In [1]:
import json
import requests
from pprint import pprint
import random
import getpass

In [2]:
OAUTH = getpass.getpass('RW Token_ID:')

RW Token_ID: ···············································································································································································································································································································································································································································································································


## Updating Widget

In [254]:
""" Copy the old one and change desired values (in this case attributes.description)
    NOTE that you cannot update id, or type, only attributes!
"""

payload = {

    
}

In [14]:
""" Copy the old one and change desired values (in this case attributes.description)
    NOTE that you cannot update id, or type, only attributes!
"""
payload = {

            "widgetConfig": {
                "interaction_config": [
                    {
                        "config": {
                            "fields": [
                                {
                                    "suffix": " Tons",
                                    "format": ".2s",
                                    "label": "Crop production",
                                    "key": "perc_value"
                                },
                                {
                                    "label": "Risk",
                                    "key": "risk"
                                },
                                {
                                    "suffix": "",
                                    "format": ".1%",
                                    "label": "Risk percentage",
                                    "key": "risk_perc"
                                }
                            ]
                        },
                        "name": "tooltip"
                    }
                ],
                "padding": "strict",
                "height": 500,
                "params_config": [
                    {
                        "required": True,
                        "key": "year"
                    },
                    {
                        "key": "iso",
                        "required": True
                    },
                    {
                        "key": "crop",
                        "required": True
                    },
                    {
                        "key": "countryName",
                        "required": True
                    }
                ],
                "data": [
                    {
                        "name": "stats",
                        "url": "https://wri-rw.carto.com/api/v2/sql?q=with a as (SELECT sum(area) as area_perc, indicator, irrigation, sum(prod) as perc_value, risk FROM crops_risk_stats WHERE year={{year}} and crop='{{crop}}' and iso='{{iso}}' GROUP BY indicator, irrigation, risk  ORDER BY indicator, indicator, risk),  b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation FROM a WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or (indicator='interannual_variability' and irrigation='irrigated') or (indicator='coastal_eutrophication_potential' and irrigation='irrigated')), c as (SELECT indicator, area_perc, perc_value, irrigation, CASE when risk IN ('No Data') then '1.No Data' when risk IN ('Low') then '2.Low' when risk IN ('Low - Medium', 'Low-Medium') then '3.Low-Mid' when risk IN ('Medium - High', 'Medium-High') then '4.Mid-High' when risk IN ('High') then '5. High' when risk IN ('Extremely High') then '6.Extremely High' when risk IN ('Arid And Low Water Use') then 'Arid and Low Water Use' END as risk,  CASE when risk IN ('No Data') then -9999 when risk IN ('Low') then 0 when risk IN ('Low - Medium', 'Low-Medium') then 1  when risk IN ('Medium - High', 'Medium-High') then 2  when risk IN ('High') then 3  when risk IN ('Extremely High') then 4 when risk IN ('Arid And Low Water Use') then 5 END as value_m FROM b) SELECT indicator, area_perc, perc_value, irrigation, risk, value_m, row_number() over (partition by indicator order by value_m asc) as index FROM c",
                        "format": {
                            "type": "json",
                            "property": "rows"
                        },
                        "transform": [
                            {
                                "type": "stack",
                                "field": "perc_value",
                                "offset": "normalize",
                                "output": {
                                    "end": "val_e",
                                    "start": "val_s"
                                },
                                "groupby": [
                                    "indicator"
                                ],
                                "sortby": [
                                    "irrigation",
                                    "indicator",
                                    "index"
                                ]
                            },
                            {
                                "by": "risk",
                                "type": "sort"
                            },
                            {
                                "expr": "datum.val_e-datum.val_s",
                                "field": "risk_perc",
                                "type": "formula"
                            }
                        ]
                    },
                    {
                        "name": "legend_data",
                        "values": [
                            {
                                "index": 1,
                                "label": "1.No Data"
                            },
                            {
                                "index": 2,
                                "label": "2.Low"
                            },
                            {
                                "index": 3,
                                "label": "3.Low-Mid"
                            },
                            {
                                "index": 4,
                                "label": "4.Mid-High"
                            },
                            {
                                "index": 5,
                                "label": "5. High"
                            },
                            {
                                "index": 6,
                                "label": "6.Extremely High"
                            },
                            {
                                "index": 7,
                                "label": "Arid & Low Water Use"
                            }
                        ],
                        "transform": [
                            {
                                "by": "index",
                                "type": "sort"
                            },
                            {
                                "expr": "length(datum.label)",
                                "field": "size",
                                "type": "formula"
                            }
                        ]
                    },
                    {
                        "name": "legend_sum",
                        "source": "legend_data",
                        "transform": [
                            {
                                "summarize": [
                                    {
                                        "field": "size",
                                        "ops": [
                                            "max"
                                        ],
                                        "as": [
                                            "max"
                                        ]
                                    },
                                    {
                                        "field": "label",
                                        "ops": [
                                            "count"
                                        ],
                                        "as": [
                                            "count"
                                        ]
                                    }
                                ],
                                "type": "aggregate"
                            }
                        ]
                    },
                    {
                        "name": "legend",
                        "source": "legend_data",
                        "transform": [
                            {
                                "output": {
                                    "right": "count",
                                    "left": "label"
                                },
                                "with": "legend_sum",
                                "diagonal": False,
                                "type": "cross"
                            },
                            {
                                "expr": "datum.label.label",
                                "field": "label_risk",
                                "type": "formula"
                            },
                            {
                                "expr": "ceil(datum.count.count/4)",
                                "field": "x_abs",
                                "type": "formula"
                            },
                            {
                                "expr": "(datum.label.index-1)%3",
                                "field": "posx",
                                "type": "formula"
                            },
                            {
                                "expr": "(ceil(datum.label.index/3))-1",
                                "field": "posy",
                                "type": "formula"
                            },
                            {
                                "expr": "datum.posy*20",
                                "field": "pos_y",
                                "type": "formula"
                            },
                            {
                                "expr": "datum.posx*datum.count.max*7",
                                "field": "pos_x",
                                "type": "formula"
                            }
                        ]
                    }
                ],
                "signals": [
                    {
                        "name": "tooltip",
                        "streams": [
                            {
                                "type": "rect:mouseover",
                                "expr": "datum"
                            },
                            {
                                "type": "rect:mouseout",
                                "expr": "{}"
                            }
                        ]
                    },
                    {
                        "name": "pos",
                        "init": {
                            "opac": 0
                        },
                        "streams": [
                            {
                                "type": "rect:mouseover",
                                "expr": "{xval: +eventX(),yval: +eventY(), opac:0.8}"
                            },
                            {
                                "type": "rect:mouseout",
                                "expr": "{opac:0}"
                            }
                        ]
                    },
                    {
                        "streams": [
                            {
                                "expr": "{ x: iscale('x', eventX()), item: eventItem() }",
                                "type": "mousemove"
                            }
                        ],
                        "name": "onMousemove"
                    }
                ],
                "scales": [
                    {
                        "name": "x",
                        "type": "linear",
                        "nice": True,
                        "domain": {
                            "data": "stats",
                            "field": "val_e"
                        },
                        "range": "width"
                    },
                    {
                        "name": "color",
                        "type": "ordinal",
                        "range": [
                            "#4E4E4E",
                            "#FFFF99",
                            "#FFE600",
                            "#FF9900",
                            "#FF1900",
                            "#990000",
                            "#dcdcdc"
                        ],
                        "domain": {
                            "data": "legend",
                            "field": "label_risk"
                        }
                    },
                    {
                        "name": "xlabels",
                        "type": "ordinal",
                        "domain": {
                            "fields": [
                                {
                                    "data": "stats",
                                    "field": "risk"
                                }
                            ]
                        },
                        "range": {
                            "fields": [
                                {
                                    "data": "stats",
                                    "field": "index"
                                }
                            ]
                        }
                    },
                    {
                        "reverse": True,
                        "name": "y_n",
                        "type": "ordinal",
                        "range": "height",
                        "domain": {
                            "data": "stats",
                            "field": "indicator"
                        }
                    },
                    {
                        "name": "legendx",
                        "type": "ordinal",
                        "domain": {
                            "data": "legend",
                            "field": "label_risk"
                        },
                        "range": {
                            "data": "legend",
                            "field": "pos_x"
                        }
                    },
                    {
                        "name": "legendy",
                        "type": "ordinal",
                        "domain": {
                            "data": "legend",
                            "field": "label_risk"
                        },
                        "range": [
                            0,
                            0,
                            0,
                            20,
                            20,
                            20,
                            40
                        ]
                    }
                ],
                "axes": [
                    {
                        "type": "y",
                        "scale": "y_n",
                        "titleOffset": 0,
                        "layer": "front",
                        "grid": False,
                        "tickSize": 1.5,
                        "properties": {
                            "axis": {
                                "stroke": {
                                    "value": "#E6E6E6"
                                },
                                "opacity": {
                                    "value": 0
                                },
                                "strokeWidth": {
                                    "value": 1.5
                                }
                            },
                            "ticks": {
                                "stroke": {
                                    "value": "#EEEEEE"
                                },
                                "strokeOpacity": {
                                    "value": 1
                                }
                            },
                            "grid": {
                                "stroke": {
                                    "value": "#EEEEEE"
                                },
                                "strokeOpacity": {
                                    "value": 1
                                }
                            },
                            "labels": {
                                "font": {
                                    "value": "\"Montserrat\", sans-serif"
                                },
                                "fill": {
                                    "value": "#9BA2AA"
                                },
                                "fontSize": {
                                    "value": 10
                                },
                                "dy": {
                                    "value": -10
                                },
                                "baseline": {
                                    "value": "middle"
                                },
                                "align": {
                                    "value": "right"
                                },
                                "angle": {
                                    "value": 0
                                },
                                "text": {
                                    "template": "{{datum.data}}"
                                }
                            }
                        }
                    },
                    {
                        "type": "x",
                        "scale": "x",
                        "titleOffset": 20,
                        "title": "% of crop production by risk",
                        "layer": "front",
                        "grid": False,
                        "format": "%",
                        "ticks": 5,
                        "tickSize": 2,
                        "properties": {
                            "axis": {
                                "stroke": {
                                    "value": "#E6E6E6"
                                },
                                "opacity": {
                                    "value": 1
                                },
                                "strokeWidth": {
                                    "value": 1.5
                                }
                            },
                            "ticks": {
                                "stroke": {
                                    "value": "#EEEEEE"
                                },
                                "strokeOpacity": {
                                    "value": 1
                                }
                            },
                            "labels": {
                                "font": {
                                    "value": "\"Montserrat\", sans-serif"
                                },
                                "fill": {
                                    "value": "#9BA2AA"
                                },
                                "fontSize": {
                                    "value": 10
                                },
                                "align": {
                                    "value": "center"
                                },
                                "baseline": {
                                    "value": "top"
                                }
                            },
                            "title": {
                                "font": {
                                    "value": "\"Montserrat\", sans-serif"
                                },
                                "fill": {
                                    "value": "#9BA2AA"
                                },
                                "fontSize": {
                                    "value": 10
                                },
                                "align": {
                                    "value": "right"
                                },
                                "baseline": {
                                    "value": "top"
                                }
                            }
                        }
                    }
                ],
                "marks": [
                    {
                        "marks": [
                            {
                                "name": "columns",
                                "type": "rect",
                                "properties": {
                                    "enter": {
                                        "x": {
                                            "scale": "x",
                                            "field": "val_s"
                                        },
                                        "x2": {
                                            "scale": "x",
                                            "field": "val_e"
                                        },
                                        "y": {
                                            "scale": "y",
                                            "field": "indicator"
                                        },
                                        "height": {
                                            "scale": "y",
                                            "band": True,
                                            "offset": -10
                                        },
                                        "fill": {
                                            "scale": "color",
                                            "field": "risk"
                                        },
                                        "fillOpacity": {
                                            "value": 1
                                        }
                                    },
                                    "update": {
                                        "fillOpacity": {
                                            "value": 1
                                        }
                                    },
                                    "hover": {
                                        "fillOpacity": {
                                            "value": 0.5
                                        }
                                    }
                                }
                            }
                        ],
                        "axes": [
                            {
                                "type": "y",
                                "scale": "y",
                                "titleOffset": 0,
                                "layer": "front",
                                "grid": False,
                                "tickSize": 1.5,
                                "properties": {
                                    "axis": {
                                        "stroke": {
                                            "value": "#E6E6E6"
                                        },
                                        "opacity": {
                                            "value": 0
                                        },
                                        "strokeWidth": {
                                            "value": 1.5
                                        }
                                    },
                                    "grid": {
                                        "stroke": {
                                            "value": "#EEEEEE"
                                        },
                                        "strokeOpacity": {
                                            "value": 1
                                        }
                                    },
                                    "ticks": {
                                        "stroke": {
                                            "value": "#EEEEEE"
                                        },
                                        "strokeOpacity": {
                                            "value": 1
                                        }
                                    },
                                    "labels": {
                                        "font": {
                                            "value": "\"Montserrat\", sans-serif"
                                        },
                                        "fill": {
                                            "value": "#9BA2AA"
                                        },
                                        "fontSize": {
                                            "value": 10
                                        },
                                        "align": {
                                            "value": "right"
                                        },
                                        "baseline": {
                                            "value": "bottom"
                                        },
                                        "dy": {
                                            "value": 10
                                        },
                                        "text": {
                                            "template": "{{datum.data}}"
                                        }
                                    }
                                }
                            }
                        ],
                        "scales": [
                            {
                                "name": "y",
                                "type": "ordinal",
                                "range": "height",
                                "domain": {
                                    "field": "irrigation"
                                }
                            }
                        ],
                        "properties": {
                            "enter": {
                                "height": {
                                    "band": True,
                                    "scale": "y_n"
                                },
                                "y": {
                                    "field": "key",
                                    "scale": "y_n"
                                }
                            }
                        },
                        "from": {
                            "transform": [
                                {
                                    "groupby": [
                                        "indicator"
                                    ],
                                    "type": "facet"
                                }
                            ],
                            "data": "stats"
                        },
                        "type": "group"
                    }
                ],
                "legends": [
                    {
                        "fill": "color",
                        "properties": {
                            "legend": {
                                "x": {
                                    "field": {
                                        "group": "width"
                                    },
                                    "mult": -0.2
                                },
                                "y": {
                                    "field": {
                                        "group": "height"
                                    },
                                    "mult": 1.2
                                }
                            },
                            "labels": {
                                "fill": {
                                    "value": "#9BA2AA"
                                },
                                "baseline": {
                                    "value": "middle"
                                },
                                "text": {
                                    "template": "{{datum.data}}"
                                },
                                "x": {
                                    "scale": "legendx",
                                    "offset": 10
                                },
                                "y": {
                                    "scale": "legendy"
                                }
                            },
                            "symbols": {
                                "stroke": {
                                    "value": "rgba(255,255,255,0)"
                                },
                                "shape": {
                                    "value": "square"
                                },
                                "size": [
                                    {
                                        "value": 100
                                    }
                                ],
                                "x": {
                                    "scale": "legendx"
                                },
                                "y": {
                                    "scale": "legendy"
                                }
                            }
                        }
                    }
                ],
                "titleConfig": {
                    "baseline": "Percentage of {{crop}} production by water risk in {{countryName}}",
                    "future": "Projected percentage of {{crop}} production by water risk in {{countryName}} in {{year}}"
                }
            },
    
}

In [15]:
#Update widget
dataset_id = '2eb9ed58-b9ba-4b47-b6a5-feed5bcf942e'
widget_id = '12362ca1-6f1a-43d3-8aaf-b25f1a22ed00'

url = f'https://staging-api.globalforestwatch.org/v1/dataset/{dataset_id}/widget/{widget_id}'

headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json'}

r = requests.patch(url, data=json.dumps(payload), headers=headers)

pprint(r.json())

{'data': {'attributes': {'application': ['aqueduct'],
                         'authors': '',
                         'createdAt': '2017-09-20T17:09:27.489Z',
                         'dataset': '2eb9ed58-b9ba-4b47-b6a5-feed5bcf942e',
                         'default': True,
                         'defaultEditableWidget': False,
                         'description': '',
                         'env': 'production',
                         'freeze': False,
                         'name': 'Percentage of crop production area by water '
                                 'risk in {{year}}',
                         'protected': False,
                         'published': True,
                         'queryUrl': 'query/2eb9ed58-b9ba-4b47-b6a5-feed5bcf942e?sql=select '
                                     '* from crops',
                         'slug': 'Percentage-of-crop-production-area-by-water-risk-in-year',
                         'source': '',
                         'templa